<a href="https://colab.research.google.com/github/vanryuji/django/blob/master/serializers/serializers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install djangorestframework psycopg2-binary
!pip list | grep djangorestframework
!pip list | grep Django

     |████████████████████████████████| 921kB 4.9MB/s 
     |████████████████████████████████| 2.9MB 32.4MB/s 
djangorestframework      3.9.4                
Django                   2.2.2                


In [0]:
from django.conf import settings
'''
When you use Django, you have to tell it which settings you’re using. Do this by using an environment variable, DJANGO_SETTINGS_MODULE.
A settings file is just a Python module with module-level variables.

In some cases, you might want to bypass the DJANGO_SETTINGS_MODULE environment variable. For example, if you’re using the template system by itself, you likely don’t want to have to set up an environment variable pointing to a settings module.
In these cases, you can configure Django’s settings manually. Do below code by calling:
'''
settings.configure(DEBUG=True)

import io
from rest_framework.parsers import JSONParser
from datetime import datetime
from rest_framework import serializers
from rest_framework.renderers import JSONRenderer

# Serializing

In [2]:
### Object -> Python native ###

class Comment(object):
    def __init__(self, email, content, created=None):
        self.email = email
        self.content = content
        self.created = created or datetime.now()

comment = Comment(email='leila@example.com', content='foo bar')
print(comment.email)

class CommentSerializer(serializers.Serializer):
    email = serializers.EmailField()
    content = serializers.CharField(max_length=200)
    created = serializers.DateTimeField()
    
    def create(self, validated_data):
        return Comment(**validated_data)

    def update(self, instance, validated_data):
        instance.email = validated_data.get('email', instance.email)
        instance.content = validated_data.get('content', instance.content)
        instance.created = validated_data.get('created', instance.created)
        return instance
    
serializer = CommentSerializer(comment)
print(serializer.data)
print(serializer.data['email'])


leila@example.com
{'email': 'leila@example.com', 'content': 'foo bar', 'created': '2019-06-20T02:21:20.976864'}
leila@example.com


In [3]:
### Python native -> JSON ###

json = JSONRenderer().render(serializer.data)
print(json)

b'{"email":"leila@example.com","content":"foo bar","created":"2019-06-20T02:21:20.976864"}'


# Deserializing

In [4]:
### JSON -> Python native ###

stream = io.BytesIO(json)
data = JSONParser().parse(stream)
print(data)
print(data['content'])

{'email': 'leila@example.com', 'content': 'foo bar', 'created': '2019-06-20T02:21:20.976864'}
foo bar


In [5]:
### Python native -> Object (.save() will create a new instance) ###

serializer = CommentSerializer(data=data)
print(serializer.is_valid())
comment1 = serializer.save()
print(comment1)
print(comment1.email)

True
leila@example.com


In [7]:
### Python native -> Object (.save() will update the existing 'comment' instance) ###

comment = Comment(email='aaa@aaa.com', content='aaaa')
print(comment.email)
serializer = CommentSerializer(comment, data=data)
print(serializer.is_valid())
comment = serializer.save()
print(comment)
print(comment.email)


aaa@aaa.com
True
leila@example.com


In [8]:
### Passing additional attributes to .save() ##

print(data)
serializer = CommentSerializer(data=data)
print(serializer.is_valid())
comment = serializer.save(email='me@google.com')
print(comment.email)

{'email': 'leila@example.com', 'content': 'foo bar', 'created': '2019-06-20T02:21:20.976864'}
True
me@google.com


# Validation

In [9]:
### Field-level validation ###

class BlogPostSerializer(serializers.Serializer):
    title = serializers.CharField(max_length=100)
    content = serializers.CharField()

    def validate_title(self, value):
        """
        Check that the blog post is about Django.
        """
        if 'django' not in value.lower():
            raise serializers.ValidationError("Blog post is not about Django")
        return value

# Validation success case
blog_post_data = {'title': 'title11_django', 'content': 'content11'}
obj = BlogPostSerializer(data=blog_post_data)
print(obj.is_valid())

# Validation failure case
blog_post_data = {'title': 'title11_', 'content': 'content11'}
obj = BlogPostSerializer(data=blog_post_data)
print(obj.is_valid())
print(obj.errors)
print(obj.is_valid(raise_exception=True))

True
False
{'title': [ErrorDetail(string='Blog post is not about Django', code='invalid')]}


ValidationError: ignored

In [10]:
### Object-level validation ###

class EventSerializer(serializers.Serializer):
    description = serializers.CharField(max_length=100)
    start = serializers.DateTimeField()
    finish = serializers.DateTimeField()

    def validate(self, data):
        """
        Check that start is before finish.
        """
        if data['start'] > data['finish']:
            raise serializers.ValidationError("finish must occur after start")
        return data
    
finish = datetime.now()
start = datetime.now()
event_data = {'description': 'desc', 'start': start, 'finish': finish}
obj = EventSerializer(data=event_data)
print(obj.is_valid())
print(obj.errors)
print(event_data['start'])
print(event_data['finish'])

False
{'non_field_errors': [ErrorDetail(string='finish must occur after start', code='invalid')]}
2019-06-20 02:24:17.591964
2019-06-20 02:24:17.591937


In [11]:
### Multiple validators ###

def v1(value):
    if 'django' not in value.lower():
        raise serializers.ValidationError("Blog post is not about Django")
        
def v2(value):
    if 'flask' not in value.lower():
        raise serializers.ValidationError("Blog post is not about Flask")


class BlogPostSerializer(serializers.Serializer):
    title = serializers.CharField(max_length=100, validators=[v1, v2])
    content = serializers.CharField()
    
blog_post_data = {'title': 'django_title', 'content': 'aaa'}
obj = BlogPostSerializer(data=blog_post_data)
print(obj.is_valid())
print(obj.errors)

blog_post_data = {'title': 'flask_title', 'content': 'aaa'}
obj = BlogPostSerializer(data=blog_post_data)
print(obj.is_valid())
print(obj.errors)

blog_post_data = {'title': 'flask_django_title', 'content': 'aaa'}
obj = BlogPostSerializer(data=blog_post_data)
print(obj.is_valid())

False
{'title': [ErrorDetail(string='Blog post is not about Flask', code='invalid')]}
False
{'title': [ErrorDetail(string='Blog post is not about Django', code='invalid')]}
True


# Accessing the initial data and instance

In [12]:
### Init CommentSerializer by instance ###

comment = Comment(email='leila@example.com', content='foo bar')
serializer = CommentSerializer(comment)
print(serializer.instance.email)

leila@example.com


In [14]:
### Init CommentSerializer by data ###

comment_data = {'email': 'aa@gmail.com', 'content': 'content', 'created': datetime.now()}
serializer = CommentSerializer(data=comment_data)
print(serializer.initial_data)
print(serializer.instance)

{'email': 'aa@gmail.com', 'content': 'content', 'created': datetime.datetime(2019, 6, 20, 2, 25, 16, 213430)}
None


# Partial updates

In [16]:
comment = Comment(email='leila@example.com', content='foo bar')
serializer = CommentSerializer(comment, data={'email': 'mac@google.com'}, partial=True)
print(serializer.is_valid())
serializer.save()
print(serializer.data)

True
{'email': 'mac@google.com', 'content': 'foo bar', 'created': '2019-06-20T02:26:04.479776'}


# Dealing with nested objects

In [25]:
class User(object):
    def __init__(self, email, username):
        self.email = email
        self.username = username
        
class Item(object):
    def __init__(self, name):
        self.name = name

class Comment(object):
    def __init__(self, user, items, content, created):
        self.user = user
        self.items = items
        self.content = content
        self.created = created
        
class UserSerializer(serializers.Serializer):
    email = serializers.EmailField()
    username = serializers.CharField(max_length=100)

class ItemSerializer(serializers.Serializer):
    name = serializers.CharField(max_length=100)

class CommentSerializer(serializers.Serializer):
    user = UserSerializer(required=False)
    items = ItemSerializer(many=True)  # A nested list of 'edit' items.
    content = serializers.CharField(max_length=200)
    created = serializers.DateTimeField()
    

user = User('aa@google.com', 'aa_user')
items = [Item('aa_item'), Item('bb_item')]
comment = Comment(user, items, 'aa_content', datetime.now())
comment_serializer = CommentSerializer(comment)
print(comment_serializer.data)
print(comment_serializer.data['items'])

{'user': OrderedDict([('email', 'aa@google.com'), ('username', 'aa_user')]), 'items': [OrderedDict([('name', 'aa_item')]), OrderedDict([('name', 'bb_item')])], 'content': 'aa_content', 'created': '2019-06-20T04:28:08.725899'}
[OrderedDict([('name', 'aa_item')]), OrderedDict([('name', 'bb_item')])]


# Including extra context

In [40]:
import requests

class Item(object):
    def __init__(self, name):
        self.name = name

class ItemSerializer(serializers.Serializer):
    name = serializers.CharField(max_length=100)
    
    def create(self, validated_data):
        print(self.context)
        return Item(**validated_data)
    
item_serializer = ItemSerializer(data={'name': 'abcde'}, context={'requests': requests})
item_serializer.is_valid()
item = item_serializer.save()
item.name

{'requests': <module 'requests' from '/usr/local/lib/python3.6/dist-packages/requests/__init__.py'>}


'abcde'